In [2]:
# RXJ0911 lens centre offset code
import math
from astropy.cosmology import Planck18 as cosmo

def extract_coordinates(filename):
    """
    Extract the (x, y) coordinates for the first and second mentioned profiles after the latest "chi^2" line.
    The first profile is labeled as G, and the second profile is labeled as G2.
    """
    x_coordinate_g = None
    y_coordinate_g = None
    x_coordinate_g2 = None
    y_coordinate_g2 = None

    try:
        with open(filename, 'r') as file:
            lines = file.readlines()

        # Find the index of the latest "chi^2" line
        latest_chi2_index = None
        for i, line in enumerate(lines):
            if "chi^2" in line:
                latest_chi2_index = i

        if latest_chi2_index is None:
            return None, None, None, None

        # Process lines after the latest "chi^2" line
        for line in lines[latest_chi2_index + 1:]:
            if "lens   sie" in line or "lens   pow" in line or "lens   anfw" in line:
                # Extract numerical values using a fixed format approach
                parts = [value for value in line.split() if value]

                try:
                    # Extract the x and y coordinates (3rd and 4th parameters)
                    x_coord = float(parts[4])  # 3rd parameter
                    y_coord = float(parts[5])  # 4th parameter

                    # Assign G and G2 based on the order of appearance
                    if x_coordinate_g is None and y_coordinate_g is None:
                        x_coordinate_g = x_coord
                        y_coordinate_g = y_coord
                    elif x_coordinate_g2 is None and y_coordinate_g2 is None:
                        x_coordinate_g2 = x_coord
                        y_coordinate_g2 = y_coord
                        break  # Stop after finding G and G2
                except (ValueError, IndexError):
                    return x_coordinate_g, y_coordinate_g, None, None

    except FileNotFoundError:
        return None, None, None, None
    except Exception:
        return None, None, None, None

    return x_coordinate_g, y_coordinate_g, x_coordinate_g2, y_coordinate_g2


def rename_file(file_path):
    """
    Rename a file based on its naming conventions.
    """
    try:
        parts = file_path.split("/")
        lens_system = parts[0]
        file_name = parts[-1]
        key = file_name.split("_optresult")[0][3:]
        profile_map = {"S": "SIE", "P": "POW", "N": "NFW"}
        profile = profile_map.get(key[0], "Unknown")
        return f"{lens_system} {profile} {key}"
    except IndexError:
        return "Unknown Description"


def calculate_offset(x, y, observed_x, observed_y):
    """
    Calculate the offset in absolute scale.
    """
    return abs(x - observed_x), abs(y - observed_y)


def calculate_total_magnitude(offset_x, offset_y):
    """
    Calculate the total magnitude of the offset using the formula:
    sqrt((x1 - x2)**2 + (y1 - y2)**2).
    """
    return math.sqrt(offset_x**2 + offset_y**2)


def calculate_physical_offset(magnitude_arcsec, redshift):
    """
    Calculate the physical offset in meters using the formula L = r * θ.
    θ is the magnitude in radians, and r is the angular diameter distance.
    """
    theta_radians = math.radians(magnitude_arcsec / 3600.0)
    r_meters = cosmo.angular_diameter_distance(redshift).to('m').value
    l_meters = r_meters * theta_radians
    l_pc = l_meters / 3.086e16
    return l_pc


def calculate_reference_scale(redshift):
    """
    Calculate the arcsecond offset for a physical distance of 10 pc.
    """
    distance_10pc_meters = 10 * 3.086e16
    r_meters = cosmo.angular_diameter_distance(redshift).to('m').value
    theta_radians = distance_10pc_meters / r_meters
    theta_arcseconds = math.degrees(theta_radians) * 3600.0
    return theta_arcseconds


def process_multiple_files(file_list, output_filename):
    """
    Process a list of files, rename them, and write the extracted results including offsets to a file,
    while also printing the results to the console.
    """
    observed_g = (0, 0)
    observed_g2 = (-0.767, 0.657)
    redshift = 0.769
    reference_scale = calculate_reference_scale(redshift)

    with open(output_filename, 'w') as output_file:
        # Write reference scale at the top
        header = f"Reference Scale: (x, y) = ({reference_scale:.4f}, {reference_scale:.4f}) arcsec for 10 pc\n\n"
        print(header.strip())
        output_file.write(header)

        for file_name in file_list:
            # Rename the file based on naming conventions
            description = rename_file(file_name)

            # Extract x and y coordinates for G and G2
            x_g, y_g, x_g2, y_g2 = extract_coordinates(file_name)

            if x_g is not None and y_g is not None:
                offset_x_g, offset_y_g = calculate_offset(x_g, y_g, observed_g[0], observed_g[1])
                total_magnitude_g = calculate_total_magnitude(offset_x_g, offset_y_g)
                physical_offset_g = calculate_physical_offset(total_magnitude_g, redshift)
                result_g = (
                    f"{description} : G (x, y) = ({x_g:.4f}, {y_g:.4f}), "
                    f"Offset = ({offset_x_g:.4f}, {offset_y_g:.4f}), "
                    f"Magnitude = {total_magnitude_g:.3f}, Physical Offset = {physical_offset_g:.0f} pc"
                )
                print(result_g)
                output_file.write(result_g + "\n")
            else:
                result_g = f"{description}: Failed to extract G (x, y)."
                print(result_g)
                output_file.write(result_g + "\n")

            if x_g2 is not None and y_g2 is not None:
                offset_x_g2, offset_y_g2 = calculate_offset(x_g2, y_g2, observed_g2[0], observed_g2[1])
                total_magnitude_g2 = calculate_total_magnitude(offset_x_g2, offset_y_g2)
                physical_offset_g2 = calculate_physical_offset(total_magnitude_g2, redshift)
                result_g2 = (
                    f"{description} : G2 (x, y) = ({x_g2:.4f}, {y_g2:.4f}), "
                    f"Offset = ({offset_x_g2:.4f}, {offset_y_g2:.4f}), "
                    f"Magnitude = {total_magnitude_g2:.3f}, Physical Offset = {physical_offset_g2:.0f} pc"
                )
                print(result_g2)
                output_file.write(result_g2 + "\n")


# List of file paths to process
file_list = [
    "RXJ0911/SPC/outSP_optresult.dat",
    "RXJ0911/SPC/outSPG_optresult.dat",
    "RXJ0911/SPC/outSPR_optresult.dat",
    "RXJ0911/SPC/outSPGR_optresult.dat",
    "RXJ0911/SPFC/outSPF_optresult.dat",
    "RXJ0911/SPFC/outSPFG_optresult.dat",
    "RXJ0911/SPFC/outSPFR_optresult.dat",
    "RXJ0911/SPFC/outSPFGR_optresult.dat",
    "RXJ0911/PPC/outPP_optresult.dat",
    "RXJ0911/PPC/outPPR_optresult.dat",
    "RXJ0911/PPC/outPPG_optresult.dat",
    "RXJ0911/PPC/outPPGR_optresult.dat",
    "RXJ0911/PPFC/outPPF_optresult.dat",
    "RXJ0911/PPFC/outPPFR_optresult.dat",
    "RXJ0911/PPFC/outPPFG_optresult.dat",
    "RXJ0911/PPFC/outPPFGR_optresult.dat",
    "RXJ0911/NPC/outNP_optresult.dat",
    "RXJ0911/NPC/outNPR_optresult.dat",
    "RXJ0911/NPC/outNPG_optresult.dat",
    "RXJ0911/NPC/outNPGR_optresult.dat",
    "RXJ0911/NPFC/outNPF_optresult.dat",
    "RXJ0911/NPFC/outNPFR_optresult.dat",
    "RXJ0911/NPFC/outNPFG_optresult.dat",
    "RXJ0911/NPFC/outNPFGR_optresult.dat",
]

# Process the files and write results to a text file
process_multiple_files(file_list, "RXJ0911_lens_centre_offset_results_gau_priored.txt")

Reference Scale: (x, y) = (0.0013, 0.0013) arcsec for 10 pc
RXJ0911 SIE SP : G (x, y) = (-0.0909, 0.0347), Offset = (0.0909, 0.0347), Magnitude = 0.097, Physical Offset = 742 pc
RXJ0911 SIE SPG : G (x, y) = (0.0224, -0.0680), Offset = (0.0224, 0.0680), Magnitude = 0.072, Physical Offset = 545 pc
RXJ0911 SIE SPG : G2 (x, y) = (-0.7703, 0.6605), Offset = (0.0033, 0.0035), Magnitude = 0.005, Physical Offset = 37 pc
RXJ0911 SIE SPR : G (x, y) = (-0.0001, 0.0316), Offset = (0.0001, 0.0316), Magnitude = 0.032, Physical Offset = 241 pc
RXJ0911 SIE SPGR : G (x, y) = (0.0000, 0.0000), Offset = (0.0000, 0.0000), Magnitude = 0.000, Physical Offset = 0 pc
RXJ0911 SIE SPGR : G2 (x, y) = (-0.7670, 0.6570), Offset = (0.0000, 0.0000), Magnitude = 0.000, Physical Offset = 0 pc
RXJ0911 SIE SPF : G (x, y) = (-0.0909, 0.0350), Offset = (0.0909, 0.0350), Magnitude = 0.097, Physical Offset = 742 pc
RXJ0911 SIE SPFG : G (x, y) = (0.0249, -0.0616), Offset = (0.0249, 0.0616), Magnitude = 0.066, Physical Offset

In [3]:
# PSJ1606 lens centre offset code
import math
from astropy.cosmology import Planck18 as cosmo

def extract_coordinates(filename):
    """
    Extract the (x, y) coordinates for the first and second mentioned profiles after the latest "chi^2" line.
    The first profile is labeled as G, and the second profile is labeled as G2.
    """
    x_coordinate_g = None
    y_coordinate_g = None
    x_coordinate_g2 = None
    y_coordinate_g2 = None

    try:
        with open(filename, 'r') as file:
            lines = file.readlines()

        # Find the index of the latest "chi^2" line
        latest_chi2_index = None
        for i, line in enumerate(lines):
            if "chi^2" in line:
                latest_chi2_index = i

        if latest_chi2_index is None:
            return None, None, None, None

        # Process lines after the latest "chi^2" line
        for line in lines[latest_chi2_index + 1:]:
            if "lens   sie" in line or "lens   pow" in line or "lens   anfw" in line:
                # Extract numerical values using a fixed format approach
                parts = [value for value in line.split() if value]

                try:
                    # Extract the x and y coordinates (3rd and 4th parameters)
                    x_coord = float(parts[4])  # 3rd parameter
                    y_coord = float(parts[5])  # 4th parameter

                    # Assign G and G2 based on the order of appearance
                    if x_coordinate_g is None and y_coordinate_g is None:
                        x_coordinate_g = x_coord
                        y_coordinate_g = y_coord
                    elif x_coordinate_g2 is None and y_coordinate_g2 is None:
                        x_coordinate_g2 = x_coord
                        y_coordinate_g2 = y_coord
                        break  # Stop after finding G and G2
                except (ValueError, IndexError):
                    return x_coordinate_g, y_coordinate_g, None, None

    except FileNotFoundError:
        return None, None, None, None
    except Exception:
        return None, None, None, None

    return x_coordinate_g, y_coordinate_g, x_coordinate_g2, y_coordinate_g2


def rename_file(file_path):
    """
    Rename a file based on its naming conventions.
    """
    try:
        parts = file_path.split("/")
        lens_system = parts[0]
        file_name = parts[-1]
        key = file_name.split("_optresult")[0][3:]
        profile_map = {"S": "SIE", "P": "POW", "N": "NFW"}
        profile = profile_map.get(key[0], "Unknown")
        return f"{lens_system} {profile} {key}"
    except IndexError:
        return "Unknown Description"


def calculate_offset(x, y, observed_x, observed_y):
    """
    Calculate the offset in absolute scale.
    """
    return abs(x - observed_x), abs(y - observed_y)


def calculate_total_magnitude(offset_x, offset_y):
    """
    Calculate the total magnitude of the offset using the formula:
    sqrt((x1 - x2)**2 + (y1 - y2)**2).
    """
    return math.sqrt(offset_x**2 + offset_y**2)


def calculate_physical_offset(magnitude_arcsec, redshift):
    """
    Calculate the physical offset in meters using the formula L = r * θ.
    θ is the magnitude in radians, and r is the angular diameter distance.
    """
    theta_radians = math.radians(magnitude_arcsec / 3600.0)
    r_meters = cosmo.angular_diameter_distance(redshift).to('m').value
    l_meters = r_meters * theta_radians
    l_pc = l_meters / 3.086e16
    return l_pc


def calculate_reference_scale(redshift):
    """
    Calculate the arcsecond offset for a physical distance of 10 pc.
    """
    distance_10pc_meters = 10 * 3.086e16
    r_meters = cosmo.angular_diameter_distance(redshift).to('m').value
    theta_radians = distance_10pc_meters / r_meters
    theta_arcseconds = math.degrees(theta_radians) * 3600.0
    return theta_arcseconds


def process_multiple_files(file_list, output_filename):
    """
    Process a list of files, rename them, and write the extracted results including offsets to a file,
    while also printing the results to the console.
    """
    observed_g = (0, 0)
    observed_g2 = (-0.307, -1.153)
    redshift = 0.3
    reference_scale = calculate_reference_scale(redshift)

    with open(output_filename, 'w') as output_file:
        # Write reference scale at the top
        header = f"Reference Scale: (x, y) = ({reference_scale:.4f}, {reference_scale:.4f}) arcsec for 10 pc\n\n"
        print(header.strip())
        output_file.write(header)

        for file_name in file_list:
            # Rename the file based on naming conventions
            description = rename_file(file_name)

            # Extract x and y coordinates for G and G2
            x_g, y_g, x_g2, y_g2 = extract_coordinates(file_name)

            if x_g is not None and y_g is not None:
                offset_x_g, offset_y_g = calculate_offset(x_g, y_g, observed_g[0], observed_g[1])
                total_magnitude_g = calculate_total_magnitude(offset_x_g, offset_y_g)
                physical_offset_g = calculate_physical_offset(total_magnitude_g, redshift)
                result_g = (
                    f"{description} : G (x, y) = ({x_g:.4f}, {y_g:.4f}), "
                    f"Offset = ({offset_x_g:.4f}, {offset_y_g:.4f}), "
                    f"Magnitude = {total_magnitude_g:.3f}, Physical Offset = {physical_offset_g:.0f} pc"
                )
                print(result_g)
                output_file.write(result_g + "\n")
            else:
                result_g = f"{description}: Failed to extract G (x, y)."
                print(result_g)
                output_file.write(result_g + "\n")

            if x_g2 is not None and y_g2 is not None:
                offset_x_g2, offset_y_g2 = calculate_offset(x_g2, y_g2, observed_g2[0], observed_g2[1])
                total_magnitude_g2 = calculate_total_magnitude(offset_x_g2, offset_y_g2)
                physical_offset_g2 = calculate_physical_offset(total_magnitude_g2, redshift)
                result_g2 = (
                    f"{description} : G2 (x, y) = ({x_g2:.4f}, {y_g2:.4f}), "
                    f"Offset = ({offset_x_g2:.4f}, {offset_y_g2:.4f}), "
                    f"Magnitude = {total_magnitude_g2:.3f}, Physical Offset = {physical_offset_g2:.0f} pc"
                )
                print(result_g2)
                output_file.write(result_g2 + "\n")


# List of file paths to process
file_list = [
    "PSJ1606/SPC/outSP_optresult.dat",
    "PSJ1606/SPC/outSPG_optresult.dat",
    "PSJ1606/SPC/outSPR_optresult.dat",
    "PSJ1606/SPC/outSPGR_optresult.dat",
    "PSJ1606/SPFC/outSPF_optresult.dat",
    "PSJ1606/SPFC/outSPFG_optresult.dat",
    "PSJ1606/SPFC/outSPFR_optresult.dat",
    "PSJ1606/SPFC/outSPFGR_optresult.dat",
    "PSJ1606/PPC/outPP_optresult.dat",
    "PSJ1606/PPC/outPPR_optresult.dat",
    "PSJ1606/PPC/outPPG_optresult.dat",
    "PSJ1606/PPC/outPPGR_optresult.dat",
    "PSJ1606/PPFC/outPPF_optresult.dat",
    "PSJ1606/PPFC/outPPFR_optresult.dat",
    "PSJ1606/PPFC/outPPFG_optresult.dat",
    "PSJ1606/PPFC/outPPFGR_optresult.dat",
    "PSJ1606/NPC/outNP_optresult.dat",
    "PSJ1606/NPC/outNPR_optresult.dat",
    "PSJ1606/NPC/outNPG_optresult.dat",
    "PSJ1606/NPC/outNPGR_optresult.dat",
    "PSJ1606/NPFC/outNPF_optresult.dat",
    "PSJ1606/NPFC/outNPFR_optresult.dat",
    "PSJ1606/NPFC/outNPFG_optresult.dat",
    "PSJ1606/NPFC/outNPFGR_optresult.dat"
]

# Process the files and write results to a text file
process_multiple_files(file_list, "PSJ1606_lens_centre_offset_results_gau_priored.txt")

Reference Scale: (x, y) = (0.0022, 0.0022) arcsec for 10 pc
PSJ1606 SIE SP : G (x, y) = (-0.0356, 0.0202), Offset = (0.0356, 0.0202), Magnitude = 0.041, Physical Offset = 188 pc
PSJ1606 SIE SPG : G (x, y) = (0.0611, 0.0535), Offset = (0.0611, 0.0535), Magnitude = 0.081, Physical Offset = 373 pc
PSJ1606 SIE SPG : G2 (x, y) = (-0.3084, -1.1403), Offset = (0.0014, 0.0127), Magnitude = 0.013, Physical Offset = 59 pc
PSJ1606 SIE SPR : G (x, y) = (-0.0169, 0.0161), Offset = (0.0169, 0.0161), Magnitude = 0.023, Physical Offset = 107 pc
PSJ1606 SIE SPGR : G (x, y) = (-0.0000, 0.0000), Offset = (0.0000, 0.0000), Magnitude = 0.000, Physical Offset = 0 pc
PSJ1606 SIE SPGR : G2 (x, y) = (-0.3070, -1.1530), Offset = (0.0000, 0.0000), Magnitude = 0.000, Physical Offset = 0 pc
PSJ1606 SIE SPF : G (x, y) = (-0.0338, 0.0149), Offset = (0.0338, 0.0149), Magnitude = 0.037, Physical Offset = 170 pc
PSJ1606 SIE SPFG : G (x, y) = (0.1040, 0.0175), Offset = (0.1040, 0.0175), Magnitude = 0.105, Physical Offse

In [1]:
# WFI2033 lens centre offset code
import math
from astropy.cosmology import Planck18 as cosmo

def extract_coordinates(filename):
    """
    Extract the (x, y) coordinates for the first and second mentioned profiles after the latest "chi^2" line.
    The first profile is labeled as G, and the second profile is labeled as G2.
    """
    x_coordinate_g = None
    y_coordinate_g = None
    x_coordinate_g2 = None
    y_coordinate_g2 = None

    try:
        with open(filename, 'r') as file:
            lines = file.readlines()

        # Find the index of the latest "chi^2" line
        latest_chi2_index = None
        for i, line in enumerate(lines):
            if "chi^2" in line:
                latest_chi2_index = i

        if latest_chi2_index is None:
            return None, None, None, None

        # Process lines after the latest "chi^2" line
        for line in lines[latest_chi2_index + 1:]:
            if "lens   sie" in line or "lens   pow" in line or "lens   anfw" in line:
                # Extract numerical values using a fixed format approach
                parts = [value for value in line.split() if value]

                try:
                    # Extract the x and y coordinates (3rd and 4th parameters)
                    x_coord = float(parts[4])  # 3rd parameter
                    y_coord = float(parts[5])  # 4th parameter

                    # Assign G and G2 based on the order of appearance
                    if x_coordinate_g is None and y_coordinate_g is None:
                        x_coordinate_g = x_coord
                        y_coordinate_g = y_coord
                    elif x_coordinate_g2 is None and y_coordinate_g2 is None:
                        x_coordinate_g2 = x_coord
                        y_coordinate_g2 = y_coord
                        break  # Stop after finding G and G2
                except (ValueError, IndexError):
                    return x_coordinate_g, y_coordinate_g, None, None

    except FileNotFoundError:
        return None, None, None, None
    except Exception:
        return None, None, None, None

    return x_coordinate_g, y_coordinate_g, x_coordinate_g2, y_coordinate_g2


def rename_file(file_path):
    """
    Rename a file based on its naming conventions.
    """
    try:
        parts = file_path.split("/")
        lens_system = parts[0]
        file_name = parts[-1]
        key = file_name.split("_optresult")[0][3:]
        profile_map = {"S": "SIE", "P": "POW", "N": "NFW"}
        profile = profile_map.get(key[0], "Unknown")
        return f"{lens_system} {profile} {key}"
    except IndexError:
        return "Unknown Description"


def calculate_offset(x, y, observed_x, observed_y):
    """
    Calculate the offset in absolute scale.
    """
    return abs(x - observed_x), abs(y - observed_y)


def calculate_total_magnitude(offset_x, offset_y):
    """
    Calculate the total magnitude of the offset using the formula:
    sqrt((x1 - x2)**2 + (y1 - y2)**2).
    """
    return math.sqrt(offset_x**2 + offset_y**2)


def calculate_physical_offset(magnitude_arcsec, redshift):
    """
    Calculate the physical offset in meters using the formula L = r * θ.
    θ is the magnitude in radians, and r is the angular diameter distance.
    """
    theta_radians = math.radians(magnitude_arcsec / 3600.0)
    r_meters = cosmo.angular_diameter_distance(redshift).to('m').value
    l_meters = r_meters * theta_radians
    l_pc = l_meters / 3.086e16
    return l_pc


def calculate_reference_scale(redshift):
    """
    Calculate the arcsecond offset for a physical distance of 10 pc.
    """
    distance_10pc_meters = 10 * 3.086e16
    r_meters = cosmo.angular_diameter_distance(redshift).to('m').value
    theta_radians = distance_10pc_meters / r_meters
    theta_arcseconds = math.degrees(theta_radians) * 3600.0
    return theta_arcseconds


def process_multiple_files(file_list, output_filename):
    """
    Process a list of files, rename them, and write the extracted results including offsets to a file,
    while also printing the results to the console.
    """
    observed_g = (0, 0)
    observed_g2 = (0.245, 2.037)
    redshift = 0.661
    reference_scale = calculate_reference_scale(redshift)

    with open(output_filename, 'w') as output_file:
        # Write reference scale at the top
        header = f"Reference Scale: (x, y) = ({reference_scale:.4f}, {reference_scale:.4f}) arcsec for 10 pc\n\n"
        print(header.strip())
        output_file.write(header)

        for file_name in file_list:
            # Rename the file based on naming conventions
            description = rename_file(file_name)

            # Extract x and y coordinates for G and G2
            x_g, y_g, x_g2, y_g2 = extract_coordinates(file_name)

            if x_g is not None and y_g is not None:
                offset_x_g, offset_y_g = calculate_offset(x_g, y_g, observed_g[0], observed_g[1])
                total_magnitude_g = calculate_total_magnitude(offset_x_g, offset_y_g)
                physical_offset_g = calculate_physical_offset(total_magnitude_g, redshift)
                result_g = (
                    f"{description} : G (x, y) = ({x_g:.4f}, {y_g:.4f}), "
                    f"Offset = ({offset_x_g:.4f}, {offset_y_g:.4f}), "
                    f"Magnitude = {total_magnitude_g:.3f}, Physical Offset = {physical_offset_g:.0f} pc"
                )
                print(result_g)
                output_file.write(result_g + "\n")
            else:
                result_g = f"{description}: Failed to extract G (x, y)."
                print(result_g)
                output_file.write(result_g + "\n")

            if x_g2 is not None and y_g2 is not None:
                offset_x_g2, offset_y_g2 = calculate_offset(x_g2, y_g2, observed_g2[0], observed_g2[1])
                total_magnitude_g2 = calculate_total_magnitude(offset_x_g2, offset_y_g2)
                physical_offset_g2 = calculate_physical_offset(total_magnitude_g2, redshift)
                result_g2 = (
                    f"{description} : G2 (x, y) = ({x_g2:.4f}, {y_g2:.4f}), "
                    f"Offset = ({offset_x_g2:.4f}, {offset_y_g2:.4f}), "
                    f"Magnitude = {total_magnitude_g2:.3f}, Physical Offset = {physical_offset_g2:.0f} pc"
                )
                print(result_g2)
                output_file.write(result_g2 + "\n")


# List of file paths to process
file_list = [
    "WFI2033/SP/outSP_optresult.dat",
    "WFI2033/SPG/outSPG_optresult.dat",
    "WFI2033/SPR/outSPR_optresult.dat",
    "WFI2033/SPGR/outSPGR_optresult.dat",
    "WFI2033/SPF/outSPF_optresult.dat",
    "WFI2033/SPFG/outSPFG_optresult.dat",
    "WFI2033/SPFR/outSPFR_optresult.dat",
    "WFI2033/SPFGR/outSPFGR_optresult.dat",
    "WFI2033/PP/outPP_optresult.dat",
    "WFI2033/PPR/outPPR_optresult.dat",
    "WFI2033/PPG/outPPG_optresult.dat",
    "WFI2033/PPGR/outPPGR_optresult.dat",
    "WFI2033/PPF/outPPF_optresult.dat",
    "WFI2033/PPFR/outPPFR_optresult.dat",
    "WFI2033/PPFG/outPPFG_optresult.dat",
    "WFI2033/PPFGR/outPPFGR_optresult.dat",
    "WFI2033/NP/outNP_optresult.dat",
    "WFI2033/NPR/outNPR_optresult.dat",
    "WFI2033/NPG/outNPG_optresult.dat",
    "WFI2033/NPGR/outNPGR_optresult.dat",
    "WFI2033/NPF/outNPF_optresult.dat",
    "WFI2033/NPFR/outNPFR_optresult.dat",
    "WFI2033/NPFG/outNPFG_optresult.dat",
    "WFI2033/NPFGR/outNPFGR_optresult.dat",
]

# Process the files and write results to a text file
process_multiple_files(file_list, "WFI2033_lens_centre_offset_results_gau_priored.txt")

/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Reference Scale: (x, y) = (0.0014, 0.0014) arcsec for 10 pc
WFI2033 SIE SP : G (x, y) = (-0.0972, 0.0428), Offset = (0.0972, 0.0428), Magnitude = 0.106, Physical Offset = 763 pc
WFI2033 SIE SPG : G (x, y) = (-0.0166, 0.0036), Offset = (0.0166, 0.0036), Magnitude = 0.017, Physical Offset = 122 pc
WFI2033 SIE SPG : G2 (x, y) = (0.2412, 2.0371), Offset = (0.0038, 0.0001), Magnitude = 0.004, Physical Offset = 27 pc
WFI2033 SIE SPR : G (x, y) = (-0.0181, 0.0352), Offset = (0.0181, 0.0352), Magnitude = 0.040, Physical Offset = 284 pc
WFI2033 SIE SPGR : G (x, y) = (-0.0012, -0.0048), Offset = (0.0012, 0.0048), Magnitude = 0.005, Physical Offset = 35 pc
WFI2033 SIE SPGR : G2 (x, y) = (0.2450, 2.0369), Offset = (0.0000, 0.0001), Magnitude = 0.000, Physical Offset = 1 pc
WFI2033 SIE SPF : G (x, y) = (-0.0474, 0.0800), Offset = (0.0474, 0.0800), Magnitude = 0.093, Physical Offset = 668 pc
WFI2033 SIE SPFG : G (x, y) = (0.0200, 0.0026), Offset = (0.0200, 0.0026), Magnitude = 0.020, Physical Offset

In [5]:
# SDSSJ1330 lens centre offset code
import math
from astropy.cosmology import Planck18 as cosmo

def extract_coordinates(filename):
    """
    Extract the (x, y) coordinates for the first and second mentioned profiles after the latest "chi^2" line.
    The first profile is labeled as G, and the second profile is labeled as G2.
    """
    x_coordinate_g = None
    y_coordinate_g = None
    x_coordinate_g2 = None
    y_coordinate_g2 = None

    try:
        with open(filename, 'r') as file:
            lines = file.readlines()

        # Find the index of the latest "chi^2" line
        latest_chi2_index = None
        for i, line in enumerate(lines):
            if "chi^2" in line:
                latest_chi2_index = i

        if latest_chi2_index is None:
            return None, None, None, None

        # Process lines after the latest "chi^2" line
        for line in lines[latest_chi2_index + 1:]:
            if "lens   sie" in line or "lens   pow" in line or "lens   anfw" in line:
                # Extract numerical values using a fixed format approach
                parts = [value for value in line.split() if value]

                try:
                    # Extract the x and y coordinates (3rd and 4th parameters)
                    x_coord = float(parts[4])  # 3rd parameter
                    y_coord = float(parts[5])  # 4th parameter

                    # Assign G and G2 based on the order of appearance
                    if x_coordinate_g is None and y_coordinate_g is None:
                        x_coordinate_g = x_coord
                        y_coordinate_g = y_coord
                    elif x_coordinate_g2 is None and y_coordinate_g2 is None:
                        x_coordinate_g2 = x_coord
                        y_coordinate_g2 = y_coord
                        break  # Stop after finding G and G2
                except (ValueError, IndexError):
                    return x_coordinate_g, y_coordinate_g, None, None

    except FileNotFoundError:
        return None, None, None, None
    except Exception:
        return None, None, None, None

    return x_coordinate_g, y_coordinate_g, x_coordinate_g2, y_coordinate_g2


def rename_file(file_path):
    """
    Rename a file based on its naming conventions.
    """
    try:
        parts = file_path.split("/")
        lens_system = parts[0]
        file_name = parts[-1]
        key = file_name.split("_optresult")[0][3:]
        profile_map = {"S": "SIE", "P": "POW", "N": "NFW"}
        profile = profile_map.get(key[0], "Unknown")
        return f"{lens_system} {profile} {key}"
    except IndexError:
        return "Unknown Description"


def calculate_offset(x, y, observed_x, observed_y):
    """
    Calculate the offset in absolute scale.
    """
    return abs(x - observed_x), abs(y - observed_y)


def calculate_total_magnitude(offset_x, offset_y):
    """
    Calculate the total magnitude of the offset using the formula:
    sqrt((x1 - x2)**2 + (y1 - y2)**2).
    """
    return math.sqrt(offset_x**2 + offset_y**2)


def calculate_physical_offset(magnitude_arcsec, redshift):
    """
    Calculate the physical offset in meters using the formula L = r * θ.
    θ is the magnitude in radians, and r is the angular diameter distance.
    """
    theta_radians = math.radians(magnitude_arcsec / 3600.0)
    r_meters = cosmo.angular_diameter_distance(redshift).to('m').value
    l_meters = r_meters * theta_radians
    l_pc = l_meters / 3.086e16
    return l_pc


def calculate_reference_scale(redshift):
    """
    Calculate the arcsecond offset for a physical distance of 10 pc.
    """
    distance_10pc_meters = 10 * 3.086e16
    r_meters = cosmo.angular_diameter_distance(redshift).to('m').value
    theta_radians = distance_10pc_meters / r_meters
    theta_arcseconds = math.degrees(theta_radians) * 3600.0
    return theta_arcseconds


def process_multiple_files(file_list, output_filename):
    """
    Process a list of files, rename them, and write the extracted results including offsets to a file,
    while also printing the results to the console.
    """
    observed_g = (0, 0)
    redshift = 0.373
    reference_scale = calculate_reference_scale(redshift)

    with open(output_filename, 'w') as output_file:
        # Write reference scale at the top
        header = f"Reference Scale: (x, y) = ({reference_scale:.4f}, {reference_scale:.4f}) arcsec for 10 pc\n\n"
        print(header.strip())
        output_file.write(header)

        for file_name in file_list:
            # Rename the file based on naming conventions
            description = rename_file(file_name)

            # Extract x and y coordinates for G and G2
            x_g, y_g, x_g2, y_g2 = extract_coordinates(file_name)

            if x_g is not None and y_g is not None:
                offset_x_g, offset_y_g = calculate_offset(x_g, y_g, observed_g[0], observed_g[1])
                total_magnitude_g = calculate_total_magnitude(offset_x_g, offset_y_g)
                physical_offset_g = calculate_physical_offset(total_magnitude_g, redshift)
                result_g = (
                    f"{description} : G (x, y) = ({x_g:.4f}, {y_g:.4f}), "
                    f"Offset = ({offset_x_g:.4f}, {offset_y_g:.4f}), "
                    f"Magnitude = {total_magnitude_g:.3f}, Physical Offset = {physical_offset_g:.0f} pc"
                )
                print(result_g)
                output_file.write(result_g + "\n")
            else:
                result_g = f"{description}: Failed to extract G (x, y)."
                print(result_g)
                output_file.write(result_g + "\n")

            if x_g2 is not None and y_g2 is not None:
                offset_x_g2, offset_y_g2 = calculate_offset(x_g2, y_g2, observed_g2[0], observed_g2[1])
                total_magnitude_g2 = calculate_total_magnitude(offset_x_g2, offset_y_g2)
                physical_offset_g2 = calculate_physical_offset(total_magnitude_g2, redshift)
                result_g2 = (
                    f"{description} : G2 (x, y) = ({x_g2:.4f}, {y_g2:.4f}), "
                    f"Offset = ({offset_x_g2:.4f}, {offset_y_g2:.4f}), "
                    f"Magnitude = {total_magnitude_g2:.3f}, Physical Offset = {physical_offset_g2:.0f} pc"
                )
                print(result_g2)
                output_file.write(result_g2 + "\n")


# List of file paths to process
file_list = [
    "SDSSJ1330/SPC/outSP_optresult.dat",
    "SDSSJ1330/SPC/outSPR_optresult.dat",
    "SDSSJ1330/SPFC/outSPF_optresult.dat",
    "SDSSJ1330/SPFC/outSPFR_optresult.dat",
    "SDSSJ1330/PPC/outPP_optresult.dat",
    "SDSSJ1330/PPC/outPPR_optresult.dat",
    "SDSSJ1330/PPFC/outPPF_optresult.dat",
    "SDSSJ1330/PPFC/outPPFR_optresult.dat",
    "SDSSJ1330/NPC/outNP_optresult.dat",
    "SDSSJ1330/NPC/outNPR_optresult.dat",
    "SDSSJ1330/NPFC/outNPF_optresult.dat",
    "SDSSJ1330/NPFC/outNPFR_optresult.dat"
]

# Process the files and write results to a text file
process_multiple_files(file_list, "SDSSJ1330_lens_centre_offset_results_gau_priored.txt")

Reference Scale: (x, y) = (0.0019, 0.0019) arcsec for 10 pc
SDSSJ1330 SIE SP : G (x, y) = (-0.0239, -0.0138), Offset = (0.0239, 0.0138), Magnitude = 0.028, Physical Offset = 146 pc
SDSSJ1330 SIE SPR : G (x, y) = (-0.0057, 0.0088), Offset = (0.0057, 0.0088), Magnitude = 0.010, Physical Offset = 55 pc
SDSSJ1330 SIE SPF : G (x, y) = (-0.0234, -0.0141), Offset = (0.0234, 0.0141), Magnitude = 0.027, Physical Offset = 145 pc
SDSSJ1330 SIE SPFR : G (x, y) = (-0.0020, 0.0084), Offset = (0.0020, 0.0084), Magnitude = 0.009, Physical Offset = 46 pc
SDSSJ1330 POW PP : G (x, y) = (0.3803, 0.4671), Offset = (0.3803, 0.4671), Magnitude = 0.602, Physical Offset = 3197 pc
SDSSJ1330 POW PPR : G (x, y) = (-0.0105, 0.0123), Offset = (0.0105, 0.0123), Magnitude = 0.016, Physical Offset = 86 pc
SDSSJ1330 POW PPF : G (x, y) = (0.3788, 0.4663), Offset = (0.3788, 0.4663), Magnitude = 0.601, Physical Offset = 3188 pc
SDSSJ1330 POW PPFR : G (x, y) = (-0.0043, 0.0163), Offset = (0.0043, 0.0163), Magnitude = 0.017

In [6]:
# WFI2026 lens centre offset code
import math
from astropy.cosmology import Planck18 as cosmo

def extract_coordinates(filename):
    """
    Extract the (x, y) coordinates for the first and second mentioned profiles after the latest "chi^2" line.
    The first profile is labeled as G, and the second profile is labeled as G2.
    """
    x_coordinate_g = None
    y_coordinate_g = None
    x_coordinate_g2 = None
    y_coordinate_g2 = None

    try:
        with open(filename, 'r') as file:
            lines = file.readlines()

        # Find the index of the latest "chi^2" line
        latest_chi2_index = None
        for i, line in enumerate(lines):
            if "chi^2" in line:
                latest_chi2_index = i

        if latest_chi2_index is None:
            return None, None, None, None

        # Process lines after the latest "chi^2" line
        for line in lines[latest_chi2_index + 1:]:
            if "lens   sie" in line or "lens   pow" in line or "lens   anfw" in line:
                # Extract numerical values using a fixed format approach
                parts = [value for value in line.split() if value]

                try:
                    # Extract the x and y coordinates (3rd and 4th parameters)
                    x_coord = float(parts[4])  # 3rd parameter
                    y_coord = float(parts[5])  # 4th parameter

                    # Assign G and G2 based on the order of appearance
                    if x_coordinate_g is None and y_coordinate_g is None:
                        x_coordinate_g = x_coord
                        y_coordinate_g = y_coord
                    elif x_coordinate_g2 is None and y_coordinate_g2 is None:
                        x_coordinate_g2 = x_coord
                        y_coordinate_g2 = y_coord
                        break  # Stop after finding G and G2
                except (ValueError, IndexError):
                    return x_coordinate_g, y_coordinate_g, None, None

    except FileNotFoundError:
        return None, None, None, None
    except Exception:
        return None, None, None, None

    return x_coordinate_g, y_coordinate_g, x_coordinate_g2, y_coordinate_g2


def rename_file(file_path):
    """
    Rename a file based on its naming conventions.
    """
    try:
        parts = file_path.split("/")
        lens_system = parts[0]
        file_name = parts[-1]
        key = file_name.split("_optresult")[0][3:]
        profile_map = {"S": "SIE", "P": "POW", "N": "NFW"}
        profile = profile_map.get(key[0], "Unknown")
        return f"{lens_system} {profile} {key}"
    except IndexError:
        return "Unknown Description"


def calculate_offset(x, y, observed_x, observed_y):
    """
    Calculate the offset in absolute scale.
    """
    return abs(x - observed_x), abs(y - observed_y)


def calculate_total_magnitude(offset_x, offset_y):
    """
    Calculate the total magnitude of the offset using the formula:
    sqrt((x1 - x2)**2 + (y1 - y2)**2).
    """
    return math.sqrt(offset_x**2 + offset_y**2)


def calculate_physical_offset(magnitude_arcsec, redshift):
    """
    Calculate the physical offset in meters using the formula L = r * θ.
    θ is the magnitude in radians, and r is the angular diameter distance.
    """
    theta_radians = math.radians(magnitude_arcsec / 3600.0)
    r_meters = cosmo.angular_diameter_distance(redshift).to('m').value
    l_meters = r_meters * theta_radians
    l_pc = l_meters / 3.086e16
    return l_pc


def calculate_reference_scale(redshift):
    """
    Calculate the arcsecond offset for a physical distance of 10 pc.
    """
    distance_10pc_meters = 10 * 3.086e16
    r_meters = cosmo.angular_diameter_distance(redshift).to('m').value
    theta_radians = distance_10pc_meters / r_meters
    theta_arcseconds = math.degrees(theta_radians) * 3600.0
    return theta_arcseconds


def process_multiple_files(file_list, output_filename):
    """
    Process a list of files, rename them, and write the extracted results including offsets to a file,
    while also printing the results to the console.
    """
    observed_g = (0, 0)
    redshift = 1.04
    reference_scale = calculate_reference_scale(redshift)

    with open(output_filename, 'w') as output_file:
        # Write reference scale at the top
        header = f"Reference Scale: (x, y) = ({reference_scale:.4f}, {reference_scale:.4f}) arcsec for 10 pc\n\n"
        print(header.strip())
        output_file.write(header)

        for file_name in file_list:
            # Rename the file based on naming conventions
            description = rename_file(file_name)

            # Extract x and y coordinates for G and G2
            x_g, y_g, x_g2, y_g2 = extract_coordinates(file_name)

            if x_g is not None and y_g is not None:
                offset_x_g, offset_y_g = calculate_offset(x_g, y_g, observed_g[0], observed_g[1])
                total_magnitude_g = calculate_total_magnitude(offset_x_g, offset_y_g)
                physical_offset_g = calculate_physical_offset(total_magnitude_g, redshift)
                result_g = (
                    f"{description} : G (x, y) = ({x_g:.4f}, {y_g:.4f}), "
                    f"Offset = ({offset_x_g:.4f}, {offset_y_g:.4f}), "
                    f"Magnitude = {total_magnitude_g:.3f}, Physical Offset = {physical_offset_g:.0f} pc"
                )
                print(result_g)
                output_file.write(result_g + "\n")
            else:
                result_g = f"{description}: Failed to extract G (x, y)."
                print(result_g)
                output_file.write(result_g + "\n")

            if x_g2 is not None and y_g2 is not None:
                offset_x_g2, offset_y_g2 = calculate_offset(x_g2, y_g2, observed_g2[0], observed_g2[1])
                total_magnitude_g2 = calculate_total_magnitude(offset_x_g2, offset_y_g2)
                physical_offset_g2 = calculate_physical_offset(total_magnitude_g2, redshift)
                result_g2 = (
                    f"{description} : G2 (x, y) = ({x_g2:.4f}, {y_g2:.4f}), "
                    f"Offset = ({offset_x_g2:.4f}, {offset_y_g2:.4f}), "
                    f"Magnitude = {total_magnitude_g2:.3f}, Physical Offset = {physical_offset_g2:.0f} pc"
                )
                print(result_g2)
                output_file.write(result_g2 + "\n")


# List of file paths to process
file_list = [
    "WFI2026/SPC/outSP_optresult.dat",
    "WFI2026/SPC/outSPR_optresult.dat",
    "WFI2026/SPFC/outSPF_optresult.dat",
    "WFI2026/SPFC/outSPFR_optresult.dat",
    "WFI2026/PPC/outPP_optresult.dat",
    "WFI2026/PPC/outPPR_optresult.dat",
    "WFI2026/PPFC/outPPF_optresult.dat",
    "WFI2026/PPFC/outPPFR_optresult.dat",
    "WFI2026/NPC/outNP_optresult.dat",
    "WFI2026/NPC/outNPR_optresult.dat",
    "WFI2026/NPFC/outNPF_optresult.dat",
    "WFI2026/NPFC/outNPFR_optresult.dat"
]

# Process the files and write results to a text file
process_multiple_files(file_list, "WFI2026_lens_centre_offset_results_gau_priored.txt")

Reference Scale: (x, y) = (0.0012, 0.0012) arcsec for 10 pc
WFI2026 SIE SP : G (x, y) = (-0.0559, 0.0028), Offset = (0.0559, 0.0028), Magnitude = 0.056, Physical Offset = 464 pc
WFI2026 SIE SPR : G (x, y) = (-0.0501, 0.0140), Offset = (0.0501, 0.0140), Magnitude = 0.052, Physical Offset = 432 pc
WFI2026 SIE SPF : G (x, y) = (-0.0517, 0.0082), Offset = (0.0517, 0.0082), Magnitude = 0.052, Physical Offset = 435 pc
WFI2026 SIE SPFR : G (x, y) = (-0.0517, 0.0198), Offset = (0.0517, 0.0198), Magnitude = 0.055, Physical Offset = 460 pc
WFI2026 POW PP : G (x, y) = (-0.0561, 0.0225), Offset = (0.0561, 0.0225), Magnitude = 0.060, Physical Offset = 502 pc
WFI2026 POW PPR : G (x, y) = (-0.0510, 0.0173), Offset = (0.0510, 0.0173), Magnitude = 0.054, Physical Offset = 447 pc
WFI2026 POW PPF : G (x, y) = (-0.0570, 0.0200), Offset = (0.0570, 0.0200), Magnitude = 0.060, Physical Offset = 501 pc
WFI2026 POW PPFR : G (x, y) = (-0.0532, 0.0176), Offset = (0.0532, 0.0176), Magnitude = 0.056, Physical Offs

In [7]:
# WGDJ0405 lens centre offset code
import math
from astropy.cosmology import Planck18 as cosmo

def extract_coordinates(filename):
    """
    Extract the (x, y) coordinates for the first and second mentioned profiles after the latest "chi^2" line.
    The first profile is labeled as G, and the second profile is labeled as G2.
    """
    x_coordinate_g = None
    y_coordinate_g = None
    x_coordinate_g2 = None
    y_coordinate_g2 = None

    try:
        with open(filename, 'r') as file:
            lines = file.readlines()

        # Find the index of the latest "chi^2" line
        latest_chi2_index = None
        for i, line in enumerate(lines):
            if "chi^2" in line:
                latest_chi2_index = i

        if latest_chi2_index is None:
            return None, None, None, None

        # Process lines after the latest "chi^2" line
        for line in lines[latest_chi2_index + 1:]:
            if "lens   sie" in line or "lens   pow" in line or "lens   anfw" in line:
                # Extract numerical values using a fixed format approach
                parts = [value for value in line.split() if value]

                try:
                    # Extract the x and y coordinates (3rd and 4th parameters)
                    x_coord = float(parts[4])  # 3rd parameter
                    y_coord = float(parts[5])  # 4th parameter

                    # Assign G and G2 based on the order of appearance
                    if x_coordinate_g is None and y_coordinate_g is None:
                        x_coordinate_g = x_coord
                        y_coordinate_g = y_coord
                    elif x_coordinate_g2 is None and y_coordinate_g2 is None:
                        x_coordinate_g2 = x_coord
                        y_coordinate_g2 = y_coord
                        break  # Stop after finding G and G2
                except (ValueError, IndexError):
                    return x_coordinate_g, y_coordinate_g, None, None

    except FileNotFoundError:
        return None, None, None, None
    except Exception:
        return None, None, None, None

    return x_coordinate_g, y_coordinate_g, x_coordinate_g2, y_coordinate_g2


def rename_file(file_path):
    """
    Rename a file based on its naming conventions.
    """
    try:
        parts = file_path.split("/")
        lens_system = parts[0]
        file_name = parts[-1]
        key = file_name.split("_optresult")[0][3:]
        profile_map = {"S": "SIE", "P": "POW", "N": "NFW"}
        profile = profile_map.get(key[0], "Unknown")
        return f"{lens_system} {profile} {key}"
    except IndexError:
        return "Unknown Description"


def calculate_offset(x, y, observed_x, observed_y):
    """
    Calculate the offset in absolute scale.
    """
    return abs(x - observed_x), abs(y - observed_y)


def calculate_total_magnitude(offset_x, offset_y):
    """
    Calculate the total magnitude of the offset using the formula:
    sqrt((x1 - x2)**2 + (y1 - y2)**2).
    """
    return math.sqrt(offset_x**2 + offset_y**2)


def calculate_physical_offset(magnitude_arcsec, redshift):
    """
    Calculate the physical offset in meters using the formula L = r * θ.
    θ is the magnitude in radians, and r is the angular diameter distance.
    """
    theta_radians = math.radians(magnitude_arcsec / 3600.0)
    r_meters = cosmo.angular_diameter_distance(redshift).to('m').value
    l_meters = r_meters * theta_radians
    l_pc = l_meters / 3.086e16
    return l_pc


def calculate_reference_scale(redshift):
    """
    Calculate the arcsecond offset for a physical distance of 10 pc.
    """
    distance_10pc_meters = 10 * 3.086e16
    r_meters = cosmo.angular_diameter_distance(redshift).to('m').value
    theta_radians = distance_10pc_meters / r_meters
    theta_arcseconds = math.degrees(theta_radians) * 3600.0
    return theta_arcseconds


def process_multiple_files(file_list, output_filename):
    """
    Process a list of files, rename them, and write the extracted results including offsets to a file,
    while also printing the results to the console.
    """
    observed_g = (0, 0)
    redshift = 0.29
    reference_scale = calculate_reference_scale(redshift)

    with open(output_filename, 'w') as output_file:
        # Write reference scale at the top
        header = f"Reference Scale: (x, y) = ({reference_scale:.4f}, {reference_scale:.4f}) arcsec for 10 pc\n\n"
        print(header.strip())
        output_file.write(header)

        for file_name in file_list:
            # Rename the file based on naming conventions
            description = rename_file(file_name)

            # Extract x and y coordinates for G and G2
            x_g, y_g, x_g2, y_g2 = extract_coordinates(file_name)

            if x_g is not None and y_g is not None:
                offset_x_g, offset_y_g = calculate_offset(x_g, y_g, observed_g[0], observed_g[1])
                total_magnitude_g = calculate_total_magnitude(offset_x_g, offset_y_g)
                physical_offset_g = calculate_physical_offset(total_magnitude_g, redshift)
                result_g = (
                    f"{description} : G (x, y) = ({x_g:.4f}, {y_g:.4f}), "
                    f"Offset = ({offset_x_g:.4f}, {offset_y_g:.4f}), "
                    f"Magnitude = {total_magnitude_g:.3f}, Physical Offset = {physical_offset_g:.0f} pc"
                )
                print(result_g)
                output_file.write(result_g + "\n")
            else:
                result_g = f"{description}: Failed to extract G (x, y)."
                print(result_g)
                output_file.write(result_g + "\n")

            if x_g2 is not None and y_g2 is not None:
                offset_x_g2, offset_y_g2 = calculate_offset(x_g2, y_g2, observed_g2[0], observed_g2[1])
                total_magnitude_g2 = calculate_total_magnitude(offset_x_g2, offset_y_g2)
                physical_offset_g2 = calculate_physical_offset(total_magnitude_g2, redshift)
                result_g2 = (
                    f"{description} : G2 (x, y) = ({x_g2:.4f}, {y_g2:.4f}), "
                    f"Offset = ({offset_x_g2:.4f}, {offset_y_g2:.4f}), "
                    f"Magnitude = {total_magnitude_g2:.3f}, Physical Offset = {physical_offset_g2:.0f} pc"
                )
                print(result_g2)
                output_file.write(result_g2 + "\n")


# List of file paths to process
file_list = [
    "WGDJ0405/SPC/outSP_optresult.dat",
    "WGDJ0405/SPC/outSPR_optresult.dat",
    "WGDJ0405/SPFC/outSPF_optresult.dat",
    "WGDJ0405/SPFC/outSPFR_optresult.dat",
    "WGDJ0405/PPC/outPP_optresult.dat",
    "WGDJ0405/PPC/outPPR_optresult.dat",
    "WGDJ0405/PPFC/outPPF_optresult.dat",
    "WGDJ0405/PPFC/outPPFR_optresult.dat",
    "WGDJ0405/NPC/outNP_optresult.dat",
    "WGDJ0405/NPC/outNPR_optresult.dat",
    "WGDJ0405/NPFC/outNPF_optresult.dat",
    "WGDJ0405/NPFC/outNPFR_optresult.dat"
]

# Process the files and write results to a text file
process_multiple_files(file_list, "WGDJ0405_lens_centre_offset_results_gau_priored.txt")

Reference Scale: (x, y) = (0.0022, 0.0022) arcsec for 10 pc
WGDJ0405 SIE SP : G (x, y) = (0.0163, -0.0095), Offset = (0.0163, 0.0095), Magnitude = 0.019, Physical Offset = 85 pc
WGDJ0405 SIE SPR : G (x, y) = (0.0135, -0.0161), Offset = (0.0135, 0.0161), Magnitude = 0.021, Physical Offset = 94 pc
WGDJ0405 SIE SPF : G (x, y) = (0.0158, -0.0113), Offset = (0.0158, 0.0113), Magnitude = 0.019, Physical Offset = 87 pc
WGDJ0405 SIE SPFR : G (x, y) = (0.0174, -0.0128), Offset = (0.0174, 0.0128), Magnitude = 0.022, Physical Offset = 97 pc
WGDJ0405 POW PP : G (x, y) = (0.0178, -0.0076), Offset = (0.0178, 0.0076), Magnitude = 0.019, Physical Offset = 87 pc
WGDJ0405 POW PPR : G (x, y) = (0.0166, -0.0133), Offset = (0.0166, 0.0133), Magnitude = 0.021, Physical Offset = 95 pc
WGDJ0405 POW PPF : G (x, y) = (0.0168, -0.0100), Offset = (0.0168, 0.0100), Magnitude = 0.020, Physical Offset = 88 pc
WGDJ0405 POW PPFR : G (x, y) = (0.0159, -0.0140), Offset = (0.0159, 0.0140), Magnitude = 0.021, Physical Off

In [9]:
# WGD2038 lens centre offset code
import math
from astropy.cosmology import Planck18 as cosmo

def extract_coordinates(filename):
    """
    Extract the (x, y) coordinates for the first and second mentioned profiles after the latest "chi^2" line.
    The first profile is labeled as G, and the second profile is labeled as G2.
    """
    x_coordinate_g = None
    y_coordinate_g = None
    x_coordinate_g2 = None
    y_coordinate_g2 = None

    try:
        with open(filename, 'r') as file:
            lines = file.readlines()

        # Find the index of the latest "chi^2" line
        latest_chi2_index = None
        for i, line in enumerate(lines):
            if "chi^2" in line:
                latest_chi2_index = i

        if latest_chi2_index is None:
            return None, None, None, None

        # Process lines after the latest "chi^2" line
        for line in lines[latest_chi2_index + 1:]:
            if "lens   sie" in line or "lens   pow" in line or "lens   anfw" in line:
                # Extract numerical values using a fixed format approach
                parts = [value for value in line.split() if value]

                try:
                    # Extract the x and y coordinates (3rd and 4th parameters)
                    x_coord = float(parts[4])  # 3rd parameter
                    y_coord = float(parts[5])  # 4th parameter

                    # Assign G and G2 based on the order of appearance
                    if x_coordinate_g is None and y_coordinate_g is None:
                        x_coordinate_g = x_coord
                        y_coordinate_g = y_coord
                    elif x_coordinate_g2 is None and y_coordinate_g2 is None:
                        x_coordinate_g2 = x_coord
                        y_coordinate_g2 = y_coord
                        break  # Stop after finding G and G2
                except (ValueError, IndexError):
                    return x_coordinate_g, y_coordinate_g, None, None

    except FileNotFoundError:
        return None, None, None, None
    except Exception:
        return None, None, None, None

    return x_coordinate_g, y_coordinate_g, x_coordinate_g2, y_coordinate_g2


def rename_file(file_path):
    """
    Rename a file based on its naming conventions.
    """
    try:
        parts = file_path.split("/")
        lens_system = parts[0]
        file_name = parts[-1]
        key = file_name.split("_optresult")[0][3:]
        profile_map = {"S": "SIE", "P": "POW", "N": "NFW"}
        profile = profile_map.get(key[0], "Unknown")
        return f"{lens_system} {profile} {key}"
    except IndexError:
        return "Unknown Description"


def calculate_offset(x, y, observed_x, observed_y):
    """
    Calculate the offset in absolute scale.
    """
    return abs(x - observed_x), abs(y - observed_y)


def calculate_total_magnitude(offset_x, offset_y):
    """
    Calculate the total magnitude of the offset using the formula:
    sqrt((x1 - x2)**2 + (y1 - y2)**2).
    """
    return math.sqrt(offset_x**2 + offset_y**2)


def calculate_physical_offset(magnitude_arcsec, redshift):
    """
    Calculate the physical offset in meters using the formula L = r * θ.
    θ is the magnitude in radians, and r is the angular diameter distance.
    """
    theta_radians = math.radians(magnitude_arcsec / 3600.0)
    r_meters = cosmo.angular_diameter_distance(redshift).to('m').value
    l_meters = r_meters * theta_radians
    l_pc = l_meters / 3.086e16
    return l_pc


def calculate_reference_scale(redshift):
    """
    Calculate the arcsecond offset for a physical distance of 10 pc.
    """
    distance_10pc_meters = 10 * 3.086e16
    r_meters = cosmo.angular_diameter_distance(redshift).to('m').value
    theta_radians = distance_10pc_meters / r_meters
    theta_arcseconds = math.degrees(theta_radians) * 3600.0
    return theta_arcseconds


def process_multiple_files(file_list, output_filename):
    """
    Process a list of files, rename them, and write the extracted results including offsets to a file,
    while also printing the results to the console.
    """
    observed_g = (0, 0)
    redshift = 0.23
    reference_scale = calculate_reference_scale(redshift)

    with open(output_filename, 'w') as output_file:
        # Write reference scale at the top
        header = f"Reference Scale: (x, y) = ({reference_scale:.4f}, {reference_scale:.4f}) arcsec for 10 pc\n\n"
        print(header.strip())
        output_file.write(header)

        for file_name in file_list:
            # Rename the file based on naming conventions
            description = rename_file(file_name)

            # Extract x and y coordinates for G and G2
            x_g, y_g, x_g2, y_g2 = extract_coordinates(file_name)

            if x_g is not None and y_g is not None:
                offset_x_g, offset_y_g = calculate_offset(x_g, y_g, observed_g[0], observed_g[1])
                total_magnitude_g = calculate_total_magnitude(offset_x_g, offset_y_g)
                physical_offset_g = calculate_physical_offset(total_magnitude_g, redshift)
                result_g = (
                    f"{description} : G (x, y) = ({x_g:.4f}, {y_g:.4f}), "
                    f"Offset = ({offset_x_g:.4f}, {offset_y_g:.4f}), "
                    f"Magnitude = {total_magnitude_g:.3f}, Physical Offset = {physical_offset_g:.0f} pc"
                )
                print(result_g)
                output_file.write(result_g + "\n")
            else:
                result_g = f"{description}: Failed to extract G (x, y)."
                print(result_g)
                output_file.write(result_g + "\n")

            if x_g2 is not None and y_g2 is not None:
                offset_x_g2, offset_y_g2 = calculate_offset(x_g2, y_g2, observed_g2[0], observed_g2[1])
                total_magnitude_g2 = calculate_total_magnitude(offset_x_g2, offset_y_g2)
                physical_offset_g2 = calculate_physical_offset(total_magnitude_g2, redshift)
                result_g2 = (
                    f"{description} : G2 (x, y) = ({x_g2:.4f}, {y_g2:.4f}), "
                    f"Offset = ({offset_x_g2:.4f}, {offset_y_g2:.4f}), "
                    f"Magnitude = {total_magnitude_g2:.3f}, Physical Offset = {physical_offset_g2:.0f} pc"
                )
                print(result_g2)
                output_file.write(result_g2 + "\n")


# List of file paths to process
file_list = [
    "WGD2038/SPC/outSP_optresult.dat",
    "WGD2038/SPC/outSPR_optresult.dat",
    "WGD2038/SPFC/outSPF_optresult.dat",
    "WGD2038/SPFC/outSPFR_optresult.dat",
    "WGD2038/PPC/outPP_optresult.dat",
    "WGD2038/PPC/outPPR_optresult.dat",
    "WGD2038/PPFC/outPPF_optresult.dat",
    "WGD2038/PPFC/outPPFR_optresult.dat",
    "WGD2038/NPC/outNP_optresult.dat",
    "WGD2038/NPC/outNPR_optresult.dat",
    "WGD2038/NPFC/outNPF_optresult.dat",
    "WGD2038/NPFC/outNPFR_optresult.dat"
]

# Process the files and write results to a text file
process_multiple_files(file_list, "WGD2038_lens_centre_offset_results_gau_priored.txt")

Reference Scale: (x, y) = (0.0026, 0.0026) arcsec for 10 pc
WGD2038 SIE SP : G (x, y) = (0.0086, 0.0052), Offset = (0.0086, 0.0052), Magnitude = 0.010, Physical Offset = 38 pc
WGD2038 SIE SPR : G (x, y) = (0.0086, 0.0052), Offset = (0.0086, 0.0052), Magnitude = 0.010, Physical Offset = 38 pc
WGD2038 SIE SPF : G (x, y) = (0.0080, 0.0042), Offset = (0.0080, 0.0042), Magnitude = 0.009, Physical Offset = 34 pc
WGD2038 SIE SPFR : G (x, y) = (0.0079, 0.0041), Offset = (0.0079, 0.0041), Magnitude = 0.009, Physical Offset = 34 pc
WGD2038 POW PP : G (x, y) = (0.0043, 0.0034), Offset = (0.0043, 0.0034), Magnitude = 0.005, Physical Offset = 21 pc
WGD2038 POW PPR : G (x, y) = (0.0000, -0.0000), Offset = (0.0000, 0.0000), Magnitude = 0.000, Physical Offset = 0 pc
WGD2038 POW PPF : G (x, y) = (0.0079, 0.0044), Offset = (0.0079, 0.0044), Magnitude = 0.009, Physical Offset = 34 pc
WGD2038 POW PPFR : G (x, y) = (0.0026, -0.0018), Offset = (0.0026, 0.0018), Magnitude = 0.003, Physical Offset = 12 pc
WGD